In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from mplsoccer import Pitch, Sbopen, VerticalPitch

In [ ]:
# load event data for match 69301
parser = Sbopen()
df, related, freeze, tactics = parser.event(69301)
#get team names
team1, team2 = df.team_name.unique()
#A dataframe of shots
shots = df.loc[df['type_name'] == 'Shot'].set_index('id')

In [ ]:
pitch = Pitch(line_color = "black")
fig, ax = pitch.draw(figsize=(10, 7))
#Size of the pitch in yards (!!!)
pitchLengthX = 120
pitchWidthY = 80
#Plot the shots by looping through them.
for i,shot in shots.iterrows():
    #get the information
    x=shot['x']
    y=shot['y']
    goal=shot['outcome_name']=='Goal'
    team_name=shot['team_name']
    #set circlesize
    circleSize=2
    #plot England
    if (team_name==team1):
        if goal:
            shotCircle=plt.Circle((x,y),circleSize,color="red")
            plt.text(x+1,y-2,shot['player_name'])
        else:
            shotCircle=plt.Circle((x,y),circleSize,color="red")
            shotCircle.set_alpha(.2)
    #plot Sweden
    else:
        if goal:
            shotCircle=plt.Circle((pitchLengthX-x,pitchWidthY - y),circleSize,color="blue")
            plt.text(pitchLengthX-x+1,pitchWidthY - y - 2 ,shot['player_name'])
        else:
            shotCircle=plt.Circle((pitchLengthX-x,pitchWidthY - y),circleSize,color="blue")
            shotCircle.set_alpha(.2)
    ax.add_patch(shotCircle)
#set title
fig.suptitle("England (red) and Sweden (blue) shots", fontsize = 24)
fig.set_size_inches(10, 7)
plt.show()

In [ ]:
#create pitch
pitch = Pitch(line_color='black')
fig, ax = pitch.grid(grid_height=0.9, title_height=0.06, axis=False,
                     endnote_height=0.04, title_space=0, endnote_space=0)
#query
mask_england = (df.type_name == 'Shot') & (df.team_name == team1)
#finding rows in the df and keeping only necessary columns
df_england = df.loc[mask_england, ['x', 'y', 'outcome_name', "player_name"]]

#plot them - if shot ended with Goal - alpha 1 and add name
#for England
for i, row in df_england.iterrows():
    if row["outcome_name"] == 'Goal':
    #make circle
       pitch.scatter(row.x, row.y, alpha = 1, s = 500, color = "red", ax=ax['pitch'])
       pitch.annotate(row["player_name"], (row.x + 1, row.y - 2), ax=ax['pitch'], fontsize = 12)
    else:
       pitch.scatter(row.x, row.y, alpha = 0.2, s = 500, color = "red", ax=ax['pitch'])

mask_sweden = (df.type_name == 'Shot') & (df.team_name == team2)
df_sweden = df.loc[mask_sweden, ['x', 'y', 'outcome_name', "player_name"]]

#for Sweden we need to revert coordinates
for i, row in df_sweden.iterrows():
    if row["outcome_name"] == 'Goal':
       pitch.scatter(120 - row.x, 80 - row.y, alpha = 1, s = 500, color = "blue", ax=ax['pitch'])
       pitch.annotate(row["player_name"], (120 - row.x + 1, 80 - row.y - 2), ax=ax['pitch'], fontsize = 12)
    else:
       pitch.scatter(120 - row.x, 80 - row.y, alpha = 0.2, s = 500, color = "blue", ax=ax['pitch'])

fig.suptitle("England (red) and Sweden (blue) shots", fontsize = 30)
plt.show()

In [ ]:
pitch = VerticalPitch(line_color='black', half = True)
fig, ax = pitch.grid(grid_height=0.9, title_height=0.06, axis=False,
                     endnote_height=0.04, title_space=0, endnote_space=0)
#plotting all shots
pitch.scatter(df_england.x, df_england.y, alpha = 1, s = 500, color = "red", ax=ax['pitch'], edgecolors="black")
fig.suptitle("England shots against Sweden", fontsize = 30)
plt.show()

#### Challenge

* Create a dataframe of passes which contains all the passes in the match

* Plot the start point of every Sweden pass. Attacking left to right.

* Plot only passes made by Caroline Seger (she is Sara Caroline Seger in the database)

* Plot arrows to show where the passes went to.


In [ ]:
df["type_name"].unique()

In [ ]:
df["sub_type_name"].unique()

In [ ]:
df["full_type_name"] = df["type_name"] + " - " + df["sub_type_name"]

In [ ]:
df["full_type_name"].unique()

In [ ]:
# include only "normal" passes, i.e., no corners, free-kicks, throw-ins etc.
df_passes = df[(df["type_name"] == "Pass") & (df["sub_type_name"].isna())]
bind_sweden = df_passes["team_name"] == team2
df_passes_sweden = df_passes[bind_sweden]

In [ ]:
pitch = Pitch(line_color='black')
fig, ax = pitch.grid(grid_height=0.9, title_height=0.06, axis=False,
                     endnote_height=0.04, title_space=0, endnote_space=0)
pitch.scatter(df_passes_sweden["x"], df_passes_sweden["y"], alpha = .4, s = 225, color = "blue", ax=ax['pitch'])

In [ ]:
df_passes_caroline_seger = df_passes_sweden[df_passes_sweden["player_name"] == "Sara Caroline Seger"]

In [ ]:
pitch = Pitch(line_color='black')
fig, ax = pitch.grid(grid_height=0.9, title_height=0.06, axis=False,
                     endnote_height=0.04, title_space=0, endnote_space=0)
pitch.scatter(df_passes_caroline_seger["x"], df_passes_caroline_seger["y"], alpha = .4, s = 225, color = "blue", ax=ax['pitch'])

In [ ]:
df_passes_caroline_seger.columns

In [ ]:
start_x = df_passes_caroline_seger["x"].to_numpy()
start_x = start_x.reshape((start_x.shape[0], 1))
start_y = df_passes_caroline_seger["y"].to_numpy()
start_y = start_y.reshape((start_y.shape[0], 1))
end_x = df_passes_caroline_seger["end_x"].to_numpy()
end_x = end_x.reshape((end_x.shape[0], 1))
end_y = df_passes_caroline_seger["end_y"].to_numpy()
end_y = end_y.reshape((end_y.shape[0], 1))

x_coord = np.concatenate((start_x, end_x), axis=1)
y_coord = np.concatenate((start_y, end_y), axis=1)

In [ ]:
pitch = Pitch(pitch_type='statsbomb', line_color='black')
fig, ax = pitch.draw(figsize=(16, 11), constrained_layout=True, tight_layout=False)
fig.set_facecolor('#22312b')
pitch.arrows(
    df_passes_caroline_seger.x,
    df_passes_caroline_seger.y,
    df_passes_caroline_seger.end_x,
    df_passes_caroline_seger.end_y,
    ax=ax,
    width=2,
    headwidth=10,
    headlength=10,
    color='blue',
    label='completed passes')

In [ ]:
x_start


In [ ]:
df_passes_caroline_seger.x